<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [1]:
#| echo: false
#| output: asis
show_doc(TopTracked2DRecording)

---

[source](https://github.com/Defense-Circuits-Lab/Gait_Analysis/blob/main/gait_analysis/twoD/cam_interfaces.py#L17){target="_blank" style="float:right; font-size:smaller"}

### TopTracked2DRecording

>      TopTracked2DRecording (filepath:pathlib.Path, week_id:int)

Very customized subclass of [`TrackedRecording`](https://Defense-Circuits-Lab.github.io/Gait_Analysis/core.html#trackedrecording) that was designed to run the gait analysis
on 2D tracking data obtained from a single camera with a top-down-view on the subject.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| filepath | Path | the filepath to the output of DLC (.h5 or .csv file) |
| week_id | int | the experimental week post injection in which the recording was performed. Has to be an element of the self.valid_week_ids list |
| **Returns** | **None** |  |

In [2]:
#| echo: false
#| output: asis
show_doc(TopTracked2DRecording.preprocess_tracking)

---

[source](https://github.com/Defense-Circuits-Lab/Gait_Analysis/blob/main/gait_analysis/twoD/cam_interfaces.py#L123){target="_blank" style="float:right; font-size:smaller"}

### TopTracked2DRecording.preprocess_tracking

>      TopTracked2DRecording.preprocess_tracking
>                                                 (marker_ids_to_compute_coverag
>                                                 e:List[str]=['TailBase',
>                                                 'Snout'],
>                                                 coverage_threshold:float=0.75,
>                                                 max_seconds_to_interpolate:flo
>                                                 at=0.5, likelihood_threshold:f
>                                                 loat=0.5, marker_ids_to_comput
>                                                 e_center_of_gravity:List[str]=
>                                                 ['TailBase', 'Snout'], relativ
>                                                 e_maze_normalization_error_tol
>                                                 erance:float=0.25)

Initiate the preprocessing of the DLC output tracking data. This includes smoothing, 
interpolation of small intervals with low likelihoods, and creation of additional markers, 
like a center of gravity. Furthermore, the code will identify the most reliable 
predicted position of the maze corners and evaluate, whether the reconstructed maze using
these most reliable open corner positions yields a maze that is sufficiently similar to how the
maze should look like (less than a specified relative error threshold). If it meets this
criterion, it will use the open corner from there on to normalize the coordinate system.
If not, it will compute everything just based on the closed corners. Either way, the
coordinate system will be normalized to the closed left corner, applying rotation and
shifting, and eventually convert the coordinates into cm values. As the very last step, 
self.bodyparts will be created - a dictionary that contains an [`Bodypart`](https://Defense-Circuits-Lab.github.io/Gait_Analysis/core.html#bodypart) object for each
marker id. Logs will save all parameters specified upon calling the function to make it
easily traceable, which parameter settings were used to obtain a given result (see also
the documentation of the `export_results()` method.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| marker_ids_to_compute_coverage | typing.List[str] | ['TailBase', 'Snout'] | List of marker_ids on base of which the tracking coverage will be computed |
| coverage_threshold | float | 0.75 | If coverage of the above defined markers is lower than this threshold, the recording will be excluded from the analysis and therefore not further processed |
| max_seconds_to_interpolate | float | 0.5 | Maximum time interval in which consecutive nan´s will be interpolated |
| likelihood_threshold | float | 0.5 | Minimum prediction likelihood of DLC that is required to acceppt predicted marker position as valid |
| marker_ids_to_compute_center_of_gravity | typing.List[str] | ['TailBase', 'Snout'] | marker ids that will be used to compute the center of gravity |
| relative_maze_normalization_error_tolerance | float | 0.25 | relative error that is tolerated when estimating the maze position for normalizing it´s position |
| **Returns** | **None** |  |  |

In [3]:
#| echo: false
#| output: asis
show_doc(TopTracked2DRecording.run_event_detection)

---

[source](https://github.com/Defense-Circuits-Lab/Gait_Analysis/blob/main/gait_analysis/twoD/cam_interfaces.py#L303){target="_blank" style="float:right; font-size:smaller"}

### TopTracked2DRecording.run_event_detection

>      TopTracked2DRecording.run_event_detection
>                                                 (bodyparts_critical_for_freezi
>                                                 ng:List[str]=['Snout',
>                                                 'CenterOfGravity'], bodyparts_
>                                                 for_direction_front_to_back:Li
>                                                 st[str]=['Snout',
>                                                 'CenterOfGravity'], immobility
>                                                 _max_rolling_speed:float=2.0, 
>                                                 immobility_min_duration:float=
>                                                 0.1, freezing_min_duration:flo
>                                                 at=0.5, gait_min_rolling_speed
>                                                 :float=3.0,
>                                                 gait_min_duration:float=0.5, g
>                                                 ait_disruption_max_time_to_imm
>                                                 obility:float=0.15, bodyparts_
>                                                 to_include_in_behavior_df=['Ce
>                                                 nterOfGravity', 'Snout',
>                                                 'TailBase'], merge_events_max_
>                                                 inbetween_time:float=0.15)

Trigger the detection of "immobility", "freezing", "gait", and "gait disruption" events. All
detected events will also be classified "towards open end" True / False, depending on whether
the orientation of the mouse was towards the open end of the maze or not. For gait events, 
this classification will be based on the orientation at the very last frame, while the orientation
at the very first frame will be used for all other events ("immobility", "freezing", and "gait
disruption"). Since the specified parameters used for this step of the analysis are very crucial,
they will be documented in the logs and exported once the `export_results()` method is called.
Default parameter selection for immobility speed threshold & freezing minimum duration was based
on: https://www.sciencedirect.com/science/article/pii/S0960982216306182

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| bodyparts_critical_for_freezing | typing.List[str] | ['Snout', 'CenterOfGravity'] | List of marker_ids that need to be classified as "immobile" that the entire mouse is classified as immobile |
| bodyparts_for_direction_front_to_back | typing.List[str] | ['Snout', 'CenterOfGravity'] | List of exactly two markers_ids that will be used to determine the orientation of the mouse; order of the markers: front, back |
| immobility_max_rolling_speed | float | 2.0 | A given bodypart will be classified as immobile in all frames, in which its rolling speed is less or equal to this value [cm per s] |
| immobility_min_duration | float | 0.1 | An immobility event will be created for each interval that exceeds this threshold [s] and in which all `bodyparts_critical_for_freezing` are classified as immobile |
| freezing_min_duration | float | 0.5 | A freezing event will be created for each interval that meets or exceeds this threshold [s] and in which all `bodyparts_critical_for_freezing` are classified as immobile |
| gait_min_rolling_speed | float | 3.0 | Frames will be classified as gait if the center of gravity meets or exceeds this rolling speed threshold [cm per s] |
| gait_min_duration | float | 0.5 | A gait event will be created for each interval that meets or exceeds this threshold [s] and in which the movement of the center of gravitiy is classified as "gait" |
| gait_disruption_max_time_to_immobility | float | 0.15 | A gait-disruption event will be created if an "immobility" event is found within this time interval [s] after a "gait" event has ended |
| bodyparts_to_include_in_behavior_df | list | ['CenterOfGravity', 'Snout', 'TailBase'] | Bodyparts that will be included in the `behavior_df`. Should include at least all `bodyparts_critical_for_freezing`, `bodyparts_for_direction_front_to_back`, and the center of gravity |
| merge_events_max_inbetween_time | float | 0.15 | Not implemented yet; |
| **Returns** | **None** |  |  |

In [4]:
#| echo: false
#| output: asis
show_doc(TopTracked2DRecording.export_results)

---

[source](https://github.com/Defense-Circuits-Lab/Gait_Analysis/blob/main/gait_analysis/twoD/cam_interfaces.py#L424){target="_blank" style="float:right; font-size:smaller"}

### TopTracked2DRecording.export_results

>      TopTracked2DRecording.export_results (out_dir_path:pathlib.Path)

Creates an .xlsx file the following tab names that contain the following information:
- parameter_settings:
    The logged parameter settings that were passed (or used by default) during the processing.

- immobility_bouts:
    Data that describe each individual immobility event that was detected.

- freezing_bouts:
    Data that describe each individual freezing event that wes detected.

- gait_bouts:
    Data that describe each individual gait event that was detected.

- gait_discruption_bouts:
    Data that describe each individual gait disruption event that was detected.

- session_overview:
    Data that summarizes all events of a given type (e.g. freezing) over the entire session.
    Events from a given type are also futher split according to the determined orientation and
    are thus always given as "towards_open_<event_type>", "towards_closed_<event_type>", or
    as "all_<event_type>" (e.g. "all_freezing_bouts", or "towards_open_gait_events").

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| out_dir_path | Path | Filepath to the directory where the results shall be saved |
| **Returns** | **None** |  |

In [5]:
#| echo: false
#| output: asis
show_doc(TopTracked2DRecording.inspect_processing)

---

[source](https://github.com/Defense-Circuits-Lab/Gait_Analysis/blob/main/gait_analysis/twoD/cam_interfaces.py#L484){target="_blank" style="float:right; font-size:smaller"}

### TopTracked2DRecording.inspect_processing

>      TopTracked2DRecording.inspect_processing
>                                                (marker_ids_to_inspect:List[str
>                                                ]=['CenterOfGravity'],
>                                                verbose:bool=False,
>                                                show_plot:bool=False,
>                                                save_plot:bool=True,
>                                                show_legend:bool=False)

Since the normalization of the maze during the preprocessing is a very critical step, this
functions allows the user to visually inspect the success of this step at a glanze. It will
plot a schematic of how the maze is expected to look like in form of a black box. It will then
also plot the "most reliable" estimate of the four maze corners after normalization and the
valid tracking positions of all markers specified in `marker_ids_to_inspect`. To avoid over-
loading of this plot, we recommend to pass only a single marker id, for instance the 
"CenterOfGravity". If you´d like to see some additional details, consider passing "verbose = True"
and/or "show_legend = True".
Note: The filepath where the plots will be saved cannot be changed & they will always be saved
in the same directory that was specified when calling the `export_results()` method, to ensure
that they are saved in the same folder as the .xlsx files that contain logged parameter settings.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| marker_ids_to_inspect | typing.List[str] | ['CenterOfGravity'] | List of marker ids whose tracked position shall be plotted as scatter plot |
| verbose | bool | False | Whether current progress and additional details like the coverages for each marker defined in `marker_ids_to_inspect` shall be printed |
| show_plot | bool | False | Whether the created plot shall be displayed in the jupyter notebook |
| save_plot | bool | True | Whether the created plot shall be saved (per default, it will be saved in the save directory that was specified when `export_results()` was called) |
| show_legend | bool | False | Whether a legend shall be added to the plot or not |
| **Returns** | **None** |  |  |